# Implementing a Custom Quadratic Layer

In this notebook, we will be focusing on creating a custom quadratic layer. This layer will perform computations based on the quadratic equation : $y = ax^2 + bx + c $ where $a$, $b$ and $c$ are the parameters that the layer will learn during training.

This specialized layer will be integrated into a model designed to work with the MNIST dataset. This exercise will give us the opportunity to explore more deeply how custom layers can be used to enhance model performance on real-world tasks. Let’s dive in and start building this exciting feature!

### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

### Define the quadratic layer
In this section, we'll build a custom layer called SimpleQuadratic to perform computations based on the quadratic formula $y=ax^2+bx+c$. This layer will have three trainable parameters: $𝑎$, $𝑏$, and $𝑐$, and will optionally allow an activation function to be applied to the output. This design provides a flexible way to integrate polynomial computations into a neural network. $ax^2 + bx + c$. We have to make sure it can also accept an activation function.

#### **Initialization (`__init__`)**
1. **Inherit from Base Class:** Begin by calling the initialization method of the base class, Layer, to ensure all underlying mechanisms are set up correctly.
2. **Units:** Define a units attribute that specifies the dimensionality of the output space (i.e., the number of output neurons).
3. **Activation Function:** Allow an activation function to be specified as a string, which can be converted into a TensorFlow object using tf.keras.activations.get(). This flexibility lets us apply any standard activation function to the output of the quadratic computations.

#### **Building the Layer (`build`)**
1. **Parameter Initialization:**
    - $a$: Initialize this weight using a normal distribution. Its shape should align with the last dimension of `input_shape` to ensure it can be appropriately multiplied with $x^2$
    - $b$: Similar to $a$, initialize using a normal distribution. while ensuring the shape matches so that $x$ can be multiplied with $b$.
    - $c$: Initialize this bias term with zeros. The length of $c$ should match the units, representing a bias for each unit.
2. **Variables as Trainable:** Mark $a$, $b$, and $c$ as trainable variables to ensure they are adjusted during the training process through backpropagation.

#### **Forward Computation (`call`)**
1. **Apply the Quadratic Formula:**
    - Compute $x^2$ and then perform a matrix multiplication with $a$ (adjust for matrix compatibility).
    - Multiply $x$ with $b$ and ensure dimensions are correct for matrix operations.
    - Sum $x^2a$. $xb$ and $c$ get the preliminary output of the layer.
2. **Activation Function:** If an activation function is specified, apply it to the output of the summation to introduce non-linearity.
3. **Output:** Return the final activated value, which is the result of the custom quadratic computations followed by the optional activation.


In [ ]:
class SimpleQuadratic(Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the class and sets up the internal variables'''
        super(SimpleQuadratic, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # a and b should be initialized with random normal, c (or the bias) with zeros and are going to be set as trainable.
        self.a = tf.Variable(name='a',
                             initial_value=tf.random_normal_initializer()(shape=(input_shape[-1], self.units),
                                                                          dtype= 'float32'), trainable = True)
        self.b = tf.Variable(name='b',
                             initial_value=tf.random_normal_initializer()(shape=(input_shape[-1], self.units),
                                                                          dtype='float32'), trainable=True)
        '''self.c = tf.Variable(name='c',
                             initial_value=tf.zeros_initializer(shape=(self.units, ),
                                                                          dtype='float32'), trainable=True)'''
        self.c = tf.Variable(name='b',
                             initial_value=tf.zeros_initializer()(shape=(self.units, ),
                                                                          dtype='float32'), trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        # Use self.activation() to get the final output
        return self.activation(tf.matmul(tf.math.square(inputs), self.a) + tf.matmul(inputs, self.b) + self.c)

Now that we have implemented our custom `SimpleQuadratic` layer, the next step is to train a model incorporating this layer. Training the model will allow us to observe how well the quadratic computations performed by the layer contribute to the overall task, such as classifying images or predicting outcomes based on input data.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  SimpleQuadratic(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.2640 - accuracy: 0.9218
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1329 - accuracy: 0.9601
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0994 - accuracy: 0.9690
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0840 - accuracy: 0.9735
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0773 - accuracy: 0.9776


[0.0772533193230629, 0.9775999784469604]